# Assignment-4: Rohitha Gutta

In [1]:
#Print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# import statements
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

In [3]:
# read edges of graph and create an graph_edges dataframe
graph_edges = spark.read.option("header", "true").csv("graph_edges.txt")

In [4]:
# display edges dataframe
graph_edges.show(5)

+-----------+-------+-----+
|edge_weight|from_id|to_id|
+-----------+-------+-----+
|          0|      5|   15|
|          1|     18|    8|
|          2|      6|    1|
|          3|      0|   10|
|          4|      2|    4|
+-----------+-------+-----+
only showing top 5 rows



In [5]:
# initial number of edges in the graph
graph_edges.count()

51

In [6]:
#cleaned the edges to ignore unwanted columns and renamed the columns names
cleaned_graph_edges = graph_edges.selectExpr("from_id as src", "to_id as dst")

In [7]:
cleaned_graph_edges.show(5)

+---+---+
|src|dst|
+---+---+
|  5| 15|
| 18|  8|
|  6|  1|
|  0| 10|
|  2|  4|
+---+---+
only showing top 5 rows



In [8]:
# read vertices of graph and create an graph_vertices dataframe
graph_vertices = spark.read.option("header", "true").csv("graph_vertices.txt")

In [9]:
# display vertices dataframe
graph_vertices.show(5)

+---------+
|vertex_id|
+---------+
|        0|
|        1|
|        2|
|        3|
|        4|
+---------+
only showing top 5 rows



In [10]:
# number of vertices in the graph
graph_vertices.count()

56

In [11]:
#renamed the column name in graph_vertices dataframe
cleaned_graph_vertices = graph_vertices.selectExpr("vertex_id as id")

In [12]:
#to convert directed graph to undirected
#make a new copy of given edges of grapfh such that the source and destination columns are swaped
graph_edges_new = cleaned_graph_edges.selectExpr( "src as dst", "dst as src")

In [13]:
#to change the order of columns in graph_edges_new to make it suitable to apply unionAll function
cleaned_graph_edges_new=graph_edges_new.select("src","dst")

In [14]:
#applied unionAll function to make graphs undirected
undirected_graph_edges = cleaned_graph_edges.unionAll(cleaned_graph_edges_new)

In [15]:
#number of edges in the graph after making graph undirected
undirected_graph_edges.count()

102

In [16]:
#create a graphframe
final_undirected_graph = GraphFrame(cleaned_graph_vertices, undirected_graph_edges)

In [17]:
#motif function to find the triangles in the given graphframe
triangles_motifs = final_undirected_graph.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")

In [18]:
triangles_motifs.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
| [5]|[15]|[31]|
|[18]| [8]|[22]|
|[17]| [8]|[22]|
|[22]| [8]|[17]|
|[22]| [8]|[18]|
|[28]| [7]|[39]|
|[31]|[15]| [5]|
|[32]|[30]|[42]|
|[39]| [7]|[28]|
|[39]|[28]| [7]|
|[42]|[32]|[30]|
| [5]|[31]|[15]|
|[30]|[42]|[32]|
|[17]|[22]| [8]|
|[18]|[22]| [8]|
|[50]|[51]|[52]|
|[51]|[52]|[50]|
|[50]|[52]|[51]|
|[71]|[72]|[73]|
|[71]|[73]|[72]|
+----+----+----+
only showing top 20 rows



In [19]:
#to find unique triangle in a graph
#covert output of motif function which is dataframe into a lists of list
triangle_list = [list(row) for row in triangles_motifs.collect()]

In [20]:
#function that returns unique traingles
def findUniqueTriangle(triangle_list):
    int_triangle_list=list()
    unique_triangle=list()
    
    for triangle in triangle_list:
        int_triangle_list.append([int(triangle[0][0]),int(triangle[1][0]), int(triangle[2][0])])
    
    for i in int_triangle_list:
        i.sort()
    
    for triangle in int_triangle_list:
        if triangle not in unique_triangle:
            unique_triangle.append(triangle)
    
    return unique_triangle

In [21]:
#unique triangles OUTPUT:
findUniqueTriangle(triangle_list)

[[5, 15, 31],
 [8, 18, 22],
 [8, 17, 22],
 [7, 28, 39],
 [30, 32, 42],
 [50, 51, 52],
 [71, 72, 73]]